In [8]:
import numpy as np
from human_moveit_config.human_model import HumanModel
import transformations
import timeit
import rospy
import time
from scipy.spatial import cKDTree
from copy import deepcopy
import tf

In [51]:
def create_database(link, base, joint_names, nb_points=1):
    X = []
    for i in range(nb_points):
        state = human.get_random_state()
        human.send_state(state) 
        fk = human.forward_kinematic(state, links=link, base='/human/' + base)
        x = []
        for j in joint_names:
            x.append(state.position[state.name.index(j)])
        x += fk[link][0]
        X.append(x)
    names = deepcopy(state.name)
    names += [link + '_x', link + '_y', link + '_z']
    np.savez_compressed('/tmp/database_' + link, names=names, data=X)

In [3]:
rospy.init_node('test')

In [4]:
human = HumanModel()

In [ ]:
tfl = tf.TransformListener()

In [52]:
links = ['shoulder_center', 'head', 'left_elbow', 'left_hand', 'right_elbow', 'right_hand']
bases = ['base', 'shoulder_center', 'shoulder_center', 'left_elbow', 'shoulder_center', 'right_elbow']
joint_by_links = {}
for l in links:
    joint_by_links[l] = human.joint_by_links[l]
for s in ['right', 'left']:
    joint_by_links[s + '_elbow'] = human.joint_by_links[s + '_shoulder'] + joint_by_links[s + '_elbow']

In [53]:
nb_points = 10
for i in range(len(links)):
    create_database(links[i], bases[i], joint_by_links[links[i]], nb_points=nb_points)

In [38]:
state = human.get_random_state()
human.send_state(state)
fk_dict = {}
for i in range(len(links)):
    fk_dict[links[i]] = human.forward_kinematic(state, links=links[i], base='/human/' + bases[i])

shoulder_center base
head shoulder_center
left_elbow shoulder_center
left_hand left_elbow
right_elbow shoulder_center
right_hand right_elbow


In [39]:
fk_dict

{'head': {'head': [[0.05529082788342368,
    -0.05155591857592216,
    0.29166287857268525],
   [0.11374117245569702,
    0.16532749504328564,
    -0.14431226049422652,
    0.9689704518416247]]},
 'left_elbow': {'left_elbow': [[-0.30053076410134644,
    0.2240666211193762,
    0.036142759508728806],
   [-0.5826834966923797,
    -0.4102918224280323,
    0.05709294061402436,
    0.6992002283059332]]},
 'left_hand': {'left_hand': [[0.2739546946062508,
    0.0034040552829809334,
    0.0998817628135768],
   [0.2608463808881614,
    -0.6384763413052102,
    0.2738510269421283,
    0.6703079458083449]]},
 'right_elbow': {'right_elbow': [[0.28965067264322114,
    -0.0831004496447179,
    -0.027082233637601316],
   [0.06298635361683091,
    -0.09945101772054202,
    0.9840597322205032,
    -0.13329912885880185]]},
 'right_hand': {'right_hand': [[0.35870716866831653,
    0.01667764088090376,
    0.04402868552006045],
   [0.021080654443818632,
    -0.22500449887012774,
    0.09086819323404415,
  

In [40]:
tfl.lookupTransform('/human/shoulder_center', '/human/head', rospy.Time(0))

((0.055290827883423654, -0.05155591857592212, 0.2916628785726854),
 (0.11374117245569691,
  0.1653274950432856,
  -0.14431226049422646,
  0.9689704518416247))

In [41]:
human.forward_kinematic(state, links='head', base='/human/shoulder_center')

{'head': [[0.05529082788342368, -0.05155591857592216, 0.29166287857268525],
  [0.11374117245569702,
   0.16532749504328564,
   -0.14431226049422652,
   0.9689704518416247]]}

In [45]:
state

header: 
  seq: 13
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: /base
name: ['spine_0', 'spine_1', 'spine_2', 'left_shoulder_0', 'left_shoulder_1', 'left_shoulder_2', 'left_elbow_0', 'left_elbow_1', 'left_wrist_0', 'left_wrist_1', 'neck_0', 'neck_1', 'neck_2', 'right_shoulder_0', 'right_shoulder_1', 'right_shoulder_2', 'right_elbow_0', 'right_elbow_1', 'right_wrist_0', 'right_wrist_1', 'left_hip_0', 'left_hip_1', 'left_hip_2', 'left_knee', 'left_ankle_0', 'left_ankle_1', 'right_hip_0', 'right_hip_1', 'right_hip_2', 'right_knee', 'right_ankle_0', 'right_ankle_1']
position: [0.9042984746069891, -0.8560502221309025, 1.0248573999126211, -0.8248428403897154, 1.369296585403056, 0.0754428557775284, -2.0865234447395764, -2.147290412407697, 0.7757021372945434, -1.5221629033442836, 0.28375957195284296, 0.29168496283144574, -0.33764962651832864, -0.18065921706669097, 1.9061405701528722, 0.8252420582987425, 3.0782930887261837, 0.43529694841374245, 0.18683445687186429, -0.4559181370589997

In [50]:
state.position[state.name.index('spine_0')]

0.9042984746069891